# Baseline with RAG Integration

This notebook extends the baseline LLM with retrieval-augmented generation (RAG).

Workflow:
- Load the same baseline model used in the prompt-engineering notebook
- Ingest a user-provided knowledge base (e.g., JSONL/CSV/TXT with recent release info)
- Build embeddings + FAISS index
- Compare answers: baseline `ask()` vs RAG `ask_rag()`



In [1]:
# Installs (Colab/Local)
%pip -q install -U transformers accelerate sentencepiece bitsandbytes sentence-transformers faiss-cpu pandas python-dotenv


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.


In [ ]:
# Optional: Hugging Face login (needed if your chosen model is gated)
from huggingface_hub import login
login(token="access token here")  # uncomment and add your token


In [15]:
import os
import json
import pandas as pd
import torch
from typing import List, Dict, Optional, Tuple
from transformers import AutoTokenizer, AutoModelForCausalLM

try:
    from transformers import BitsAndBytesConfig
    _bnb_available = True
except Exception:
    _bnb_available = False

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_ID = "google/gemma-2-2b-it"  # same baseline

# Generation defaults
GEN_CFG = {
    "max_new_tokens": 800,
    "temperature": 0.3,
    "top_p": 0.9,
    "repetition_penalty": 1.1,
}

print("Device:", DEVICE)
print("Model:", MODEL_ID)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)

if DEVICE == "cuda" and _bnb_available:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float16,
    )
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        device_map="auto",
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
    )
else:
    dtype = torch.float32 if DEVICE == "cpu" else torch.float16
    model = AutoModelForCausalLM.from_pretrained(MODEL_ID, torch_dtype=dtype)
    model.to(DEVICE)

model.eval()
print("Model loaded.")


Device: cuda
Model: google/gemma-2-2b-it


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded.


In [16]:
PYTHON_ASSISTANT_SYSTEM_PROMPT = "You are a Python programming assistant."

# Chat formatting with system-role fallback for templates that don't support 'system'
def _format_chat(messages: List[Dict[str, str]], add_generation_prompt: bool = True) -> Dict[str, torch.Tensor]:
    if hasattr(tokenizer, "apply_chat_template") and tokenizer.chat_template is not None:
        effective_messages = messages
        if messages and messages[0].get("role") == "system":
            system_text = messages[0]["content"]
            effective_messages = messages[1:]
            if effective_messages and effective_messages[0].get("role") == "user":
                effective_messages = effective_messages.copy()
                effective_messages[0] = {
                    "role": "user",
                    "content": f"{system_text}\n\n{effective_messages[0]['content']}"
                }
            else:
                effective_messages = [{"role": "user", "content": system_text}]
        prompt_text = tokenizer.apply_chat_template(
            effective_messages,
            tokenize=False,
            add_generation_prompt=add_generation_prompt
        )
    else:
        sys_msg = ""
        if messages and messages[0].get("role") == "system":
            sys_msg = f"System: {messages[0]['content']}\n"
            user_msgs = messages[1:]
        else:
            user_msgs = messages
        convo = "\n".join([f"{m['role'].capitalize()}: {m['content']}" for m in user_msgs])
        prompt_text = (sys_msg + convo + ("\nAssistant:" if add_generation_prompt else ""))

    inputs = tokenizer(prompt_text, return_tensors="pt")
    return {k: v.to(DEVICE) for k, v in inputs.items()}

@torch.inference_mode()
def generate_from_messages(
    messages: List[Dict[str, str]],
    max_new_tokens: int = GEN_CFG["max_new_tokens"],
    temperature: float = GEN_CFG["temperature"],
    top_p: float = GEN_CFG["top_p"],
    repetition_penalty: float = GEN_CFG["repetition_penalty"],
) -> str:
    inputs = _format_chat(messages, add_generation_prompt=True)
    input_len = inputs["input_ids"].shape[-1]
    outputs = model.generate(
        **inputs,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.eos_token_id,
    )
    gen_ids = outputs[0][input_len:]
    text = tokenizer.decode(gen_ids, skip_special_tokens=True)
    return text.strip()

def ask(question: str, system_prompt: Optional[str] = PYTHON_ASSISTANT_SYSTEM_PROMPT, **gen_kwargs) -> str:
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": question})
    return generate_from_messages(messages, **gen_kwargs)


In [17]:
# Knowledge base ingestion
# Accepts: JSONL (one object per line with fields: id, content, [title|version|urls...])
#          CSV (column 'content' required)
#          TXT/MD (each file is one document)
from pathlib import Path
from typing import Union

def read_jsonl(path: Union[str, Path]) -> List[Dict[str, str]]:
    records = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            obj = json.loads(line)
            content = obj.get("content") or obj.get("text") or obj.get("body")
            if not content:
                continue
            records.append({
                "id": obj.get("id") or obj.get("_id") or str(len(records)),
                "title": obj.get("title") or "",
                "content": content,
                "meta": {k: v for k, v in obj.items() if k not in {"id", "_id", "title", "content", "text", "body"}}
            })
    return records

def read_csv(path: Union[str, Path]) -> List[Dict[str, str]]:
    df = pd.read_csv(path)
    if "content" not in df.columns:
        raise ValueError("CSV must have a 'content' column")
    recs = []
    for i, row in df.iterrows():
        recs.append({
            "id": str(row.get("id", i)),
            "title": str(row.get("title", "")),
            "content": str(row["content"]),
            "meta": {k: row[k] for k in df.columns if k not in {"id", "title", "content"}}
        })
    return recs

def read_texts(paths: List[Union[str, Path]]) -> List[Dict[str, str]]:
    recs = []
    for p in paths:
        p = Path(p)
        with open(p, "r", encoding="utf-8") as f:
            text = f.read()
        recs.append({
            "id": p.stem,
            "title": p.name,
            "content": text,
            "meta": {"path": str(p)}
        })
    return recs

def load_kb(path_or_dir: Union[str, Path]) -> List[Dict[str, str]]:
    p = Path(path_or_dir)
    if p.is_dir():
        files = list(p.rglob("*.jsonl")) + list(p.rglob("*.csv")) + list(p.rglob("*.txt")) + list(p.rglob("*.md"))
        recs: List[Dict[str, str]] = []
        for fpath in files:
            if fpath.suffix == ".jsonl":
                recs.extend(read_jsonl(fpath))
            elif fpath.suffix == ".csv":
                recs.extend(read_csv(fpath))
            else:
                recs.extend(read_texts([fpath]))
        return recs
    else:
        if p.suffix == ".jsonl":
            return read_jsonl(p)
        if p.suffix == ".csv":
            return read_csv(p)
        if p.suffix in {".txt", ".md"}:
            return read_texts([p])
        raise ValueError("Unsupported KB format; use .jsonl, .csv, .txt, .md or a directory containing them")


In [18]:
# Embeddings + FAISS index
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

EMBED_MODEL_ID = "sentence-transformers/all-MiniLM-L6-v2"  # small, fast

class VectorStore:
    def __init__(self, dim: int):
        self.dim = dim
        self.index = faiss.IndexFlatIP(dim)  # cosine via normalized dot product
        self.docs: List[Dict[str, str]] = []

    def add(self, embeddings: np.ndarray, docs: List[Dict[str, str]]):
        # Normalize for cosine similarity
        faiss.normalize_L2(embeddings)
        self.index.add(embeddings)
        self.docs.extend(docs)

    def search(self, query_emb: np.ndarray, k: int = 5) -> List[Tuple[int, float]]:
        faiss.normalize_L2(query_emb)
        D, I = self.index.search(query_emb, k)
        return list(zip(I[0].tolist(), D[0].tolist()))

# Build embeds
embedder = SentenceTransformer(EMBED_MODEL_ID)
emb_dim = embedder.get_sentence_embedding_dimension()
store = VectorStore(dim=emb_dim)

# Helper to (re)build index from records
def build_index(records: List[Dict[str, str]], batch_size: int = 64):
    texts = [r["content"] for r in records]
    embeddings = []
    for i in range(0, len(texts), batch_size):
        chunk = texts[i:i+batch_size]
        vecs = embedder.encode(chunk, convert_to_numpy=True, show_progress_bar=False, normalize_embeddings=False)
        embeddings.append(vecs)
    if not embeddings:
        return
    embeddings = np.vstack(embeddings)
    store.add(embeddings, records)

print("Embedding model:", EMBED_MODEL_ID)


Embedding model: sentence-transformers/all-MiniLM-L6-v2


In [19]:
# Retrieval and RAG ask()
from textwrap import dedent

def embed(texts: List[str]) -> np.ndarray:
    vecs = embedder.encode(texts, convert_to_numpy=True, show_progress_bar=False, normalize_embeddings=False)
    return vecs

def retrieve(query: str, k: int = 4) -> List[Dict[str, str]]:
    qvec = embed([query])
    hits = store.search(qvec, k=k)
    results = []
    for idx, score in hits:
        if idx == -1:
            continue
        doc = store.docs[idx]
        results.append({"score": float(score), **doc})
    return results

def make_context(snippets: List[Dict[str, str]]) -> str:
    blocks = []
    for s in snippets:
        title = s.get("title") or s.get("id") or ""
        prefix = f"Title: {title}\n" if title else ""
        blocks.append(prefix + s["content"])
    return "\n\n---\n\n".join(blocks)

def answer_with_context(question: str, context: str) -> str:
    sys = PYTHON_ASSISTANT_SYSTEM_PROMPT
    prompt = dedent(f"""
    Use the context below to answer the user question. If the answer is not in the context, say you don't know.

    Context:
    {context}

    Question: {question}
    """)
    return ask(prompt, system_prompt=sys, max_new_tokens=GEN_CFG["max_new_tokens"], temperature=GEN_CFG["temperature"])

# Public API

def ask_baseline(question: str) -> str:
    return ask(question)

def ask_rag(question: str, k: int = 4) -> Tuple[str, List[Dict[str, str]]]:
    snippets = retrieve(question, k=k)
    context = make_context(snippets)
    answer = answer_with_context(question, context)
    return answer, snippets


In [21]:
# Demo: Load your KB, build index, compare baseline vs RAG
# 1) Point to a path or upload files via the Colab file browser
KB_PATH = "/content/updated_python_kb.jsonl"  # change to your folder or a specific .jsonl/.csv/.txt

# Example: you can also mount Google Drive in Colab and set KB_PATH accordingly

records = load_kb(KB_PATH)
print(f"Loaded {len(records)} records from KB.")

# 2) Build (or rebuild) the FAISS index
store = VectorStore(dim=emb_dim)
build_index(records)
print(f"Indexed {len(store.docs)} documents.")

# 3) Ask a question without context (baseline) vs with RAG
question = "What were the documented release blockers and notable open issues listed before the Python 3.13.1 release, and which were resolved by that release?"


print("\nQ:", question)
base_answer = ask_baseline(question)
rag_answer, used = ask_rag(question, k=4)
print("\nBaseline:\n", base_answer)
print("\nRAG:\n", rag_answer)
print("\nContext snippets used (top-4):")
for i, snip in enumerate(used, 1):
        print(f"[{i}] id={snip.get('id')} title={snip.get('title','')[:80]} score={snip.get('score'):.3f}")


Loaded 7 records from KB.
Indexed 7 documents.

Q: What were the documented release blockers and notable open issues listed before the Python 3.13.1 release, and which were resolved by that release?

Baseline:
 I do not have access to real-time information, including specific details about software releases like the Python 3.13.1 release notes.  My knowledge is based on the data I was trained on, which has a cutoff point. 

**To find the information you're looking for, I recommend checking these resources:**

* **The official Python website:** https://www.python.org/
* **The Python 3.13.1 Release Notes:** You can usually find this on the official Python website or in the release announcement.
* **GitHub:** The Python project's GitHub repository might have detailed release notes: https://github.com/python/cpython


These sources will provide the most accurate and up-to-date information regarding documented release blockers and open issues addressed in the Python 3.13.1 release.

RAG:
 A